In [1]:
import json
import pandas as pd
import numpy as np
import PySAM.Pvwattsv8 as PVWatts
import PySAM.Grid as Grid
import csv
import matplotlib.pyplot as plt
from datetime import datetime
from matplotlib.dates import DateFormatter
import geopandas as gpd
import math

In [2]:
def loadModules(file_names):
    pv = PVWatts.new()
    grid = Grid.from_existing(pv)

    with open(file_names[0], 'r') as file:
        data = json.load(file)
        for k, v in data.items():
                pv.value(k, v)

    with open(file_names[1], 'r') as file:
        data = json.load(file)
        for k, v in data.items():
                grid.value(k, v)

    return pv, grid

file_names = ["/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/Scripts/sunEstimation/pysam_template_pvwattsv8.json",
    "/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/Scripts/sunEstimation/pysam_template_grid.json"]

loadModules(file_names)

(<Pvwattsv8 at 0x7eff90052820>, <Grid at 0x7eff90051d40>)

In [5]:
planeFile = "/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/Results/Test_70_el Besòs i el Maresme/Parcels/4157903DF3845E/86/Shading/13.csv"
shadingProfilesDF = pd.read_csv(planeFile, header=None)
tilts = shadingProfilesDF.iloc[0][3:363]


def get_matrix(tilts):
    tilts = tilts.round()
    angles = np.unique(tilts)
    azimuths = np.arange(0, 360, 1)

    matrix = []
    singleRow = []
    singleRow.append(0)
    for angle in azimuths:
        singleRow.append(angle)

    matrix.append(singleRow)
    for i, angle in enumerate(angles):
        singleRow = []
        singleRow.append(angle)
        for j, tilt in enumerate(tilts):
            if angle <= tilt:
                singleRow.append(100)
            else:
                singleRow.append(0)
        matrix.append(singleRow)

    # matrix = np.zeros((len(angles) + 1, len(azimuths) + 1), dtype=int)
    # matrix[0, 1:] = azimuths
    # matrix[1:, 0] = angles

    # for i, angle in enumerate(angles):
    #     for j, tilt in enumerate(tilts):
    #         if angle <= tilt:
    #             matrix[i + 1, j + 1] = 100

    return matrix

shadingMatrix = get_matrix(tilts)

In [7]:
planeIdGDF = gpd.read_file("/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/Results/Test_70_el Besòs i el Maresme/Parcels/4157903DF3845E/86/Plane Identification/86.gpkg")
plane = planeIdGDF[planeIdGDF.cluster == 13]

def getInfoRoof(plane):
    tilt = plane.tilt.values[0]
    azimuth = plane.azimuth.values[0]
    area = plane.area.values[0]/math.cos(tilt*math.pi/180)
    return  area, tilt, azimuth
    
area, tilt, azimuth = getInfoRoof(plane)
plane

,cluster,A,B,D,tilt,azimuth,silhouette,geometry
1,13,-0.115791,0.407912,-1.820217e+06,22.978,164.152739,-0.001242,"MULTIPOLYGON (((434060.3 4585522.628, 434060.3..."


In [10]:
def runPySAMSimulation(file_names, tilts, plane, tmyfile, exportPath):
    pv, grid = loadModules(file_names)
    shadingMatrix = get_matrix(tilts)

    area, tilt, azimuth = getInfoRoof(plane)
    
    ratio=0.400/(1.879*1.045)

    modifiedParams = {"shading_azal": shadingMatrix,
        "system_capacity": area*ratio, #*self.pv.value("gcr"), #We don't need the area by the ground coverage ratio
        "tilt": tilt,
        "azimuth": azimuth,
        "solar_resource_file": tmyfile}

    for i in range(len(modifiedParams)): 
        pv.value(list(modifiedParams.keys())[i], list(modifiedParams.values())[i])

    modules = [pv, grid]
    
    for m in modules:
        m.execute()

    generation = pv.export()["Outputs"]["ac"]
    generation = np.array(generation).reshape(365, 24)
    generation_df = pd.DataFrame(generation)
    
    # generation_df.to_csv((exportPath + '.csv'), index=False)
    return generation_df/area

tmyfile = "/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/RAW_Data/TMY/NREL/419806_41.41_2.22_tmy-2022.csv"
generation_df = runPySAMSimulation(file_names, tilts, plane, tmyfile, "exportPath")

In [11]:
generation_df.max()

0       0.000000
1       0.000000
2       0.000000
3       0.000000
4       0.000000
5      12.316267
6      51.356979
7      91.611809
8     122.625420
9     146.708007
10    162.003923
11    166.493721
12    161.551630
13    146.043757
14    122.725653
15     92.710843
16     53.958421
17     26.957041
18      9.423263
19      0.000000
20      0.000000
21      0.000000
22      0.000000
23      0.000000
dtype: float64

# As a function

In [12]:
import json
import pandas as pd
import numpy as np
import PySAM.Pvwattsv8 as PVWatts
import PySAM.Grid as Grid
import csv
import os
import geopandas as gpd
import math

In [13]:
def loadModules(file_names):
    pv = PVWatts.new()
    grid = Grid.from_existing(pv)

    with open(file_names[0], 'r') as file:
        data = json.load(file)
        for k, v in data.items():
                pv.value(k, v)

    with open(file_names[1], 'r') as file:
        data = json.load(file)
        for k, v in data.items():
                grid.value(k, v)

    return pv, grid

def get_matrix(tilts):
    tilts = tilts.round()
    angles = np.unique(tilts)
    azimuths = np.arange(0, 360, 1)

    matrix = []
    singleRow = []
    singleRow.append(0)
    for angle in azimuths:
        singleRow.append(angle)

    matrix.append(singleRow)
    for i, angle in enumerate(angles):
        singleRow = []
        singleRow.append(angle)
        for j, tilt in enumerate(tilts):
            if angle <= tilt:
                singleRow.append(100)
            else:
                singleRow.append(0)
        matrix.append(singleRow)
    return matrix


def runPySAMSimulation(file_names, tilts, plane, tmyfile, exportPath):
    pv, grid = loadModules(file_names)
    shadingMatrix = get_matrix(tilts)

    area, tilt, azimuth = getInfoRoof(plane)
    
    ratio=float(0.450/2)

    modifiedParams = {"shading_azal": shadingMatrix,
        "system_capacity": area*ratio, #*self.pv.value("gcr"), #We don't need the area by the ground coverage ratio
        "tilt": tilt,
        "azimuth": azimuth,
        "solar_resource_file": tmyfile}

    for i in range(len(modifiedParams)): 
        pv.value(list(modifiedParams.keys())[i], list(modifiedParams.values())[i])

    modules = [pv, grid]
    
    for m in modules:
        m.execute()

    generation = pv.export()["Outputs"]["ac"]
    generation = np.array(generation).reshape(365, 24)
    generation_df = pd.DataFrame(generation)
    
    # generation_df.to_csv((exportPath + '.csv'), index=False)
    return generation_df/area



In [14]:
file_names = ["/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/Scripts/sunEstimation/pysam_template_pvwattsv8.json",
    "/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/Scripts/sunEstimation/pysam_template_grid.json"]

tmyfile = "/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/RAW_Data/TMY/NREL/419806_41.41_2.22_tmy-2022.csv"

basePath = "/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/"
neighborhood = "Test_70_el Besòs i el Maresme"
parcelsFolder = basePath + "/Results/" + neighborhood + "/Parcels/"

annual_ac = 0

for parcel in os.listdir(parcelsFolder):
    parcelSubfolder = parcelsFolder + parcel + "/"
    for construction in [x for x in os.listdir(parcelSubfolder) if os.path.isdir(parcelSubfolder + x)]:
        constructionFolder = parcelSubfolder + construction + "/"
        planesGDF = gpd.read_file(constructionFolder + "Plane Identification/" + construction + ".gpkg")
        for cluster in planesGDF.cluster.values:
            shadingFile = constructionFolder + "/Shading/" + str(cluster) + ".csv"
            plane = planesGDF[planesGDF.cluster == cluster]
            if os.path.isfile(shadingFile):
                if(os.stat(shadingFile).st_size > 0):
                    shadingProfilesDF = pd.read_csv(shadingFile, header=None)
                    tilts = shadingProfilesDF.iloc[0][3:363]

                    generation_df = runPySAMSimulation(file_names, tilts, plane, tmyfile, "exportPath")
                    annual_ac = generation_df.sum().sum()        
print(annual_ac)

272938.68342764745


In [36]:
shadingFile

'/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3//Results/Test_70_el Besòs i el Maresme/Parcels/4151302DF3845A/139//Shading/12.csv'